In [23]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [24]:
train_data = pd.read_csv('train.csv')

In [25]:
y1_train = train_data['label'].values
X1_train = train_data.drop(columns=['label']).values/255
X_test = np.array(X1_train[3600:4200])
y_test = np.array(y1_train[3600:4200])
X_train = np.array(X1_train[0:3600])
y_train = np.array(y1_train[0:3600])



In [26]:
def relu(x):  #defined relu function
    x[x<0]=0
    return x

In [27]:
def softmax(X_in,weights):  # defined softmax function
    s = np.exp(np.matmul(X_in,weights))
    total = np.sum(s, axis=1).reshape(-1,1)
    return s / total

In [28]:
def loss(y_pred,y_true):    # defined loss function
    global K 
    K = 10
    N = len(y_true)
    y_true_one_hot_vec = (y_true[:,np.newaxis] == np.arange(K))
    loss_sample = (np.log(y_pred) * y_true_one_hot_vec).sum(axis=1)
    return -np.mean(loss_sample)

In [29]:
def h(X,W,b):      #forward propogation
    a1 = X
    z1 = np.matmul(X, W[0]) + b[0]
    a2 = relu(z1)
    z2 = np.matmul(a2, W[1]) + b[1]
    s = np.exp(z2)
    total = np.sum(s, axis=1).reshape(-1,1)
    sigma = s/total
    return sigma

In [30]:
def backprop(W,b,X,y,alpha=1e-4):    #backward propogation
    a1 = X
    z1 = np.matmul(X, W[0]) + b[0]
    a2 = relu(z1)
    z2 = np.matmul(a2, W[1]) + b[1]
    s = np.exp(z2)
    total = np.sum(s, axis=1).reshape(-1,1)
    sigma = s/total
    
    K = 10
    N = X.shape[0]
    y_one_hot_vec = (y[:,np.newaxis] == np.arange(K))
    delta2 = (sigma - y_one_hot_vec)
    grad_W1 = np.matmul(a2.T, delta2)
    
    delta1 = np.matmul(delta2, W[1].T)*(z1>0)
    grad_W0 = np.matmul(X.T, delta1)
    
    dW = [grad_W0/N + alpha*W[0], grad_W1/N + alpha*W[1]]
    db0 = np.mean(delta1, axis=0)
    db1 = np.mean(delta2, axis=0)
    return dW,db0,db1

In [31]:
num_iter = 2200 # number of iterations of gradient descent
alpha = 0.04
n_H = 256 # number of neurons in the hidden layer
n = X_train.shape[1] # number of pixels in an image
K = 10
alpha = 0.001
# np.random.seed(1127)
W = [1e-1*np.random.randn(n, n_H),1e-1*np.random.randn(n_H, K)]
b = [np.random.randn(n_H), np.random.randn(K)]

In [32]:
for i in range(num_iter):
    dW,db0,db1 = backprop(W,b,X_train,y_train,alpha)
    W[0] = W[0] - alpha*(dW[0])
    W[1] = W[1] - alpha*(dW[1])
    b[0] = b[0] - alpha*(db0)
    b[1] = b[1] - alpha*(db1)
    y_pred_final = h(X_train,W,b)
    if(i%100 == 0):
        print("Final cross-entropy loss is {:.8}".format(loss(y_pred_final,y_train)))
y_pred_final = h(X_train,W,b)
print(db1.shape)
print("Final cross-entropy loss is {:.8}".format(loss(y_pred_final,y_train)))
print("Final training accuracy is {:.4%}".format(np.mean(np.argmax(y_pred_final, axis=1)== y_train)))
print(y_pred_final)

Final cross-entropy loss is 3.5621791
Final cross-entropy loss is 2.2910176
Final cross-entropy loss is 2.0214507
Final cross-entropy loss is 1.8609841
Final cross-entropy loss is 1.7249816
Final cross-entropy loss is 1.6072377
Final cross-entropy loss is 1.504924
Final cross-entropy loss is 1.4155471
Final cross-entropy loss is 1.3370822
Final cross-entropy loss is 1.2679283
Final cross-entropy loss is 1.2066258
Final cross-entropy loss is 1.1519715
Final cross-entropy loss is 1.103031
Final cross-entropy loss is 1.0590393
Final cross-entropy loss is 1.0193262
Final cross-entropy loss is 0.9832956
Final cross-entropy loss is 0.95049423
Final cross-entropy loss is 0.92051146
Final cross-entropy loss is 0.89302039
Final cross-entropy loss is 0.86775095
Final cross-entropy loss is 0.84442347
Final cross-entropy loss is 0.82283401
(10,)
Final cross-entropy loss is 0.80301253
Final training accuracy is 80.5278%
[[2.67140006e-03 6.84206643e-01 4.72521178e-02 ... 7.56238810e-03
  1.20952840e

In [33]:
sigma = h(X_test,W,b)
sigma_new = np.argmax(sigma,axis = 1)
print("test data accuracy = ",np.sum((sigma_new == y_test))/sigma_new.shape[0] * 100)

test data accuracy =  79.66666666666666
